In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import codecs
from os import listdir
import codecs
import re
import string
import pickle
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
language = 'PERSIAN'
path_prefix = ''
if language == 'PERSIAN':
    path_prefix = 'English files/'

In [ ]:
def load_docs(given_path):
    docs = []
    counter = 0
    for fileName in listdir(given_path):
        counter += 1
        file = open(given_path + fileName, 'r', encoding='utf-8')
        text = file.read()
        docs.append(text.split())
        file.close()
        if language == 'ENGLISH' and counter == 50000: break
        print(counter)
    return docs

formal_male_data = None
formal_female_data = None
if language == 'PERSIAN':
    formal_male_data = load_docs('Gender tagged corpus cleaned/NEW/formal language/male/')
    formal_female_data = load_docs('Gender tagged corpus cleaned/NEW/formal language/female/')
    print('PERSIAN docs loaded')
else:
    formal_male_data = load_docs('Gender tagged corpus English/male/')
    formal_female_data = load_docs('Gender tagged corpus English/female/')
    print('ENGLISH docs loaded')

all_data = formal_male_data + formal_female_data
all_labels = [1] * len(formal_male_data) + [0] * len(formal_female_data)
print(len(all_data))
print(len(formal_male_data), len(formal_female_data))

In [ ]:
features = 300
tokenizer = Tokenizer(num_words = features)
tokenizer.fit_on_texts(all_data)
with open(path_prefix + 'GenderTokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Tokenizer saved.")
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(all_data)
max_length = max([len(sent) for sent in X])
vocab_size = len(tokenizer.word_index) + 1
X = pad_sequences(X, padding='post')
y = all_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
print(len(X_train), len(X_val), len(X_test))
print(vocab_size, max_length)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train[:40000], y_train[:40000])

In [ ]:
predictions = model.predict(X_test[:10000])

In [ ]:
rights = 0
for i, each in enumerate(predictions):
    if y_test[i] == each:
        rights += 1
print('Logistic Regression accuracy: %.2f' % ((rights * 100) / len(y_test)))